# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_before.png" alt="Image 1" style="width:300px; border:1px solid #ccc; border-radius:6px;">
  <img src="./assets/LC_Memory.png" alt="Image 2" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [3]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [4]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [5]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_wkvLIfRCADfV85rVhwY0rjAE)
 Call ID: call_wkvLIfRCADfV85rVhwY0rjAE
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' AND (name LIKE '%invoice%' OR name LIKE '%customer%' OR name LIKE '%client%' OR name LIKE '%bill%') ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Customer',), ('Invoice',), ('InvoiceLine',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_aVTA8A7w6oHXhn6qyJ374T3n)
 Call ID: call_aVTA8A7w6oHXhn6qyJ374T3n
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Customer c
JOIN Invoice i ON i.CustomerId = c.CustomerId
WHERE c.FirstName = 'Frank' AND c.L

In [8]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you clarify which titles you mean? For example:
- Titles of books, movies, articles, or something else?
- From which table, and with any filters?

If you want, I can list the first 5 titles from a specified table.


## Add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

In [21]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [22]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_w9WWXMQSbBZ2cdzs89EgH89Z)
 Call ID: call_w9WWXMQSbBZ2cdzs89EgH89Z
  Args:
    query: SELECT name FROM sqlite_master WHERE type = 'table' ORDER BY name LIMIT 50;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_QvnKDPh17i3rIg2jVmZULt0b)
 Call ID: call_QvnKDPh17i3rIg2jVmZULt0b
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice AS i
JOIN Customer AS c ON c.CustomerId = i.CustomerId
WHERE c.Fir

In [23]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_GnE4SCBOiTl1eSVeIE17C2Ay)
 Call ID: call_GnE4SCBOiTl1eSVeIE17C2Ay
  Args:
    query: SELECT t.Name AS Title
FROM InvoiceLine AS il
JOIN Track AS t ON t.TrackId = il.TrackId
WHERE il.InvoiceId = 374
ORDER BY il.InvoiceLineId ASC
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

Here are the track titles on that invoice:
- Holier Than Thou
- Through The Never
- My Friend Of Misery
- The Wait
- Blitzkrieg

Note: I’ve listed the first 5. If you want the full list, just say “all titles.”


## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)